In [1]:
import os, re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
os.chdir('..')

---

In [4]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'ibes_summary_file_raw.tsv'),
                 sep = '\t')

keep relevant columns

In [6]:
df = df[['ticker', 'fpedats', 'statpers', 'anndats_act', 'actual', 'meanest', 'medest', 'numest']]

delete:

1. if actual EPS is missing
2. if forecast data is after annual report date
3. forecast is not the latest forecast (i.e., only keep latest forecast)

In [7]:
df = df.loc[df['actual'].notna()]

In [8]:
df['statpers'] = pd.to_datetime(df['statpers'])
df['anndats_act'] = pd.to_datetime(df['anndats_act'])

In [9]:
df = df.loc[df['statpers'] == df.groupby(['ticker', 'fpedats'])['statpers'].transform('max')]

sort

In [10]:
df = df.sort_values(by = ['ticker', 'fpedats', 'statpers'])

get fyear

In [11]:
df['fpedats'] = pd.to_datetime(df['fpedats'])

In [12]:
df['fyear'] = np.where(df['fpedats'].dt.month < 6,
                       df['fpedats'].dt.year - 1,
                       df['fpedats'].dt.year)

save to disk

In [13]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'ibes_summary_file_clean.tsv'),
          sep = '\t', index = False)